In [1]:
import cv2
import joblib
import numpy as np
import matplotlib.pyplot as plt
from skimage.feature import hog
from skimage import img_as_float
from skimage.metrics import peak_signal_noise_ratio as psnr
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split

In [2]:
def start_edit_image(path):
    """
    Начало обработки изображения. оттенки серого -> понижение шума -> 
    адаптивная бинаризация -> выделение контуров -> расширение линий.
    """
    image = cv2.imread(path)
    image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    blurred = cv2.GaussianBlur(image_gray, (9, 9), 5)
    
    binary = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                   cv2.THRESH_BINARY_INV, 17, 1)
    
    edges = cv2.Canny(binary, 50, 150, 3)
    
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    dilated_edges = cv2.dilate(edges, kernel, iterations=1)
    return dilated_edges, image

In [3]:
def search_contours(dilated_edges, original_image):
    """
    Поиск контура сетки судоку. Находит самый большой контур, выполняет аппроксимацию линий.
    """
    contours, hierarchy = cv2.findContours(dilated_edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    largest_contour = max(contours, key=cv2.contourArea)
    
    epsilon = 0.02 * cv2.arcLength(largest_contour, True)
    approx = cv2.approxPolyDP(largest_contour, epsilon, True)

    image_copy = original_image.copy()
    cv2.drawContours(image_copy, [approx], -1, (0, 255, 0), 1)
    
    cv2.imshow("Sudoku Grid", image_copy)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    return approx
    

In [4]:
def expand_rect(rect, scale=1.05):
    """
    Расширяет или сжимает прямоугольник, изменяя его размеры относительно центра.
    """
    center_x, center_y = np.mean(rect[:, 0]), np.mean(rect[:, 1])
    
    expanded_rect = []
    for x, y in rect:
        new_x = center_x + scale * (x - center_x) 
        new_y = center_y + scale * (y - center_y)
        expanded_rect.append([new_x, new_y])
    return np.array(expanded_rect, dtype="float32")

def order_points(pts):
    """
    Упорядочивает точки в заданной последовательности.
    """
    rect = np.zeros((4, 2), dtype="float32")
    s = pts.sum(axis=1)
    rect[0] = pts[np.argmin(s)]  # Верхний левый
    rect[2] = pts[np.argmax(s)]  # Нижний правый

    diff = np.diff(pts, axis=1)
    rect[1] = pts[np.argmin(diff)]  # Верхний правый
    rect[3] = pts[np.argmax(diff)]  # Нижний левый

    return rect

In [5]:
def correct_image(image, approx, max_width=450):
    """
    Преобразует сетку судоку в 'ровное' изображение.
    """
    points = approx.reshape(4, 2)
    rect = order_points(points)

    max_height = max_width
    
    expanded_rect = expand_rect(rect, scale=0.98)  # 1.1 означает увеличение на 10%
    
    dst = np.array([
        [0, 0],
        [max_width - 1, 0],
        [max_width - 1, max_height - 1],
        [0, max_height - 1]
    ], dtype="float32")
    
    M = cv2.getPerspectiveTransform(expanded_rect, dst)
    warped = cv2.warpPerspective(image, M, (max_width, max_height))
    
    cv2.imshow("Expanded Sudoku Grid", warped)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    return warped

In [6]:
def get_cells(warped):
    """
    Извлечь ячейки из сетки судоку, разделив ее на равные части 9*9.
    """
    cell_height = warped.shape[0] // 9
    cell_width = warped.shape[1] // 9

    cells = []
    for row in range(9):
        for col in range(9):
            x1 = col * cell_width
            y1 = row * cell_height
            x2 = x1 + cell_width
            y2 = y1 + cell_height
            cell = warped[y1:y2, x1:x2]
            cells.append(cell)

    return cells

In [7]:
def crop_cell_by_percent(cell, crop_percent=0.1):
    """
    Обрезает ячейку, чтобы убрать лишние объекты (линни по краям).
    """
    if not (0 <= crop_percent < 0.5):
        raise ValueError("crop_percent должен быть в диапазоне от 0 до 0.5")
    
    height, width = cell.shape[:2]

    top = int(height * crop_percent)
    bottom = int(height * (1 - crop_percent))
    left = int(width * crop_percent)
    right = int(width * (1 - crop_percent))

    cropped = cell[top:bottom, left:right]
    return cropped

In [8]:
def edit_cell(cell):
    """
    Обрабатывает ячейку.
    """
    warped_gray = cv2.cvtColor(crop_cell_by_percent(cell, 0.2), cv2.COLOR_BGR2GRAY)
    
    blurred_part = cv2.GaussianBlur(warped_gray, (5, 5), 3)
    
    binary = cv2.adaptiveThreshold(blurred_part, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                   cv2.THRESH_BINARY_INV, 7, 1.7)
    
    kernel = np.ones((2, 2), np.uint8)
    
    eroded = cv2.erode(binary, kernel, iterations=1)
    
    border_size = int(eroded.shape[0] * 0.15)
    
    bordered_image = cv2.copyMakeBorder(
    eroded, 
    border_size, border_size, border_size, border_size,  # Верх, низ, лево, право
    cv2.BORDER_CONSTANT, 
    value=0 
    )
    
    return bordered_image

In [9]:
def simpling_massive(massive):
    """
    Преобразует активность пикселей в 0 и 1 (1 - если пиксель активен).
    """
    new_massive = massive.copy()
    for i in range(len(new_massive)):
        for j in range(len(new_massive[i])):
            if new_massive[i][j]:
                new_massive[i][j] = 1
    return new_massive

In [10]:
def prepare_list_cells(cells):
    """
    Формирует список, подходящий под модель.
    """
    cells_edited = []
    for i in cells:
        cells_edited.append(edit_cell(i))

    cells_resized = []
    for i in cells_edited:
        cells_resized.append(cv2.resize(i, (28, 28), interpolation=cv2.INTER_AREA).reshape(-1))

    simple_cells_resized = simpling_massive(cells_resized)
    return simple_cells_resized, cells_resized


In [11]:
def detect_none_figure(massive):
    """
    Выявляет пустые ячейки на сетке.
    """
    null_massive = []
    for i in massive:
        count_pixel = 0
        for j in i:
            if j:
                count_pixel += 1
        if count_pixel > 40:
            null_massive.append(1)
        else:
            null_massive.append(0)
    return null_massive

In [12]:
def main_function(path):
    dilated_edges, image = start_edit_image(path)
    approx = search_contours(dilated_edges, image)
    warped = correct_image(image, approx)
    cells = get_cells(warped)
    simple_cells_resized, cells_resized = prepare_list_cells(cells)
    null_massive = detect_none_figure(cells_resized)

    model = joblib.load("mnist_model_simple.pkl")
    
    y_pred = model.predict(simple_cells_resized)

    for i in range(len(y_pred)):
        if not null_massive[i]:
            y_pred[i] = '0'
    
    return y_pred


In [14]:
main_function('./sudoku_dataset/image126.jpg').reshape(9, 9)

array([['5', '0', '1', '0', '0', '5', '0', '0', '4'],
       ['0', '0', '0', '2', '0', '0', '0', '0', '0'],
       ['0', '0', '8', '0', '0', '0', '0', '2', '0'],
       ['0', '1', '0', '0', '0', '0', '0', '0', '0'],
       ['2', '0', '2', '0', '5', '0', '0', '0', '9'],
       ['0', '0', '0', '4', '6', '0', '1', '0', '0'],
       ['0', '2', '0', '0', '2', '0', '4', '0', '0'],
       ['0', '0', '0', '0', '0', '0', '7', '4', '0'],
       ['0', '0', '1', '2', '0', '0', '0', '4', '0']], dtype=object)